In [1]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from itertools import combinations

In [2]:

file_path = 'CEP paper review form (Responses) 20250618.xlsx'
df = pd.read_excel(file_path)

df.columns = df.columns.str.strip()
#print(df.columns)

#cleaning up and creating Year
df.rename(columns={
    'Who are you': 'Rater',
    'Nature (Simon) see classification details': 'Nature',
    'Context (Simon) see classification details. Use "other" if necessary': 'Context',
    'Theme/topic (Simon) see classification details. Use "other" if necessary': 'Theme',
    'Scope (Simon) see classification details': 'Scope',
    'Title of paper (for cross-checking only)': 'Title',
    'UK institutions contributing data/context, select at least one ("no data/context" is an option, only use if there is absolutely nothing: no module titles, no description of year groups etc). Some institutions are titled "University Of..."': 'Institutions'
}, inplace=True)

df['Year'] = df['Which paper'].astype(str).str[:4]

df

,Timestamp,Rater,Which paper,Are you primary or secondary reviewer,"Section headings, top-level only. One per line",Number of references in the paper,Institutions,Title,Nature,Scope,Context,Theme,Column 16,Notes and queries,C,Year
0,2025-06-02 15:09:09.846,Rose,2019_3294016.3294017,Secondary,NaN,NaN,NaN,Teaching Data Ethics,Report,Subject,ethics/professionalism,curriculum,NaN,NaN,NaN,2019
1,2025-06-12 14:11:19.497,Mark,2019_3294016.3294017,Primary,Introduction\nBackground\nEvaluation\nFuture W...,21.0,University of St Andrews,Teaching Data Ethics,Report,Subject,ethics/professionalism,curriculum,NaN,NaN,NaN,2019
2,2025-05-26 13:24:32.849,Mark,2019_3294016.3294018,Primary,Introduction\nOur Experiences with Codio\nValu...,6.0,Coventry University,Computing with Codio at Coventry University,Report,Program/department,Programming,curriculum,NaN,NaN,NaN,2019
3,2025-06-02 15:10:34.650,Rose,2019_3294016.3294018,Secondary,NaN,NaN,NaN,Computing with Codio at Coventry University,Report,Subject,Programming,teaching/learning tools,NaN,NaN,NaN,2019
4,2025-05-26 13:31:24.541,Mark,2019_3294016.3294019,Primary,Introduction\nBackground\nContext\nPractice\nA...,39.0,Falmouth University,Nurturing Collaboration in an Undergraduate Co...,Analysis,Program/department,groupwork,competitions,NaN,NaN,NaN,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,2025-06-16 16:37:41.610,Duncan,2023_3573260.3573265,Secondary,NaN,NaN,NaN,Data Science Course Design for a Large-Scale C...,Report,Subject,data mining,"tutors, demonstrators, mentors",NaN,NaN,NaN,2023
162,2025-06-16 16:39:07.291,Duncan,2023_3573260.3573266,Secondary,NaN,NaN,NaN,Introducing Modelling and Code Comprehension f...,Report,Subject,programming languages,teaching/learning theories & models,NaN,NaN,NaN,2023
163,2025-06-16 16:40:58.899,Duncan,2023_3573260.3573267,Secondary,NaN,NaN,NaN,Exploring Student Perceptions and Expectations...,Report,Institution,security,attitudes,NaN,NaN,NaN,2023
164,2025-06-16 16:42:31.505,Duncan,2023_3573260.3573268,Secondary,NaN,NaN,NaN,An alternative supervision model for postgradu...,Report,Program/department,capstone project,"tutors, demonstrators, mentors",NaN,NaN,NaN,2023


In [3]:
# Identifying "other" values

originalOptions = {"Context": 
                         ["artificial intelligence", "broad-based", "capstone project", "communication skills", "compilers", "cryptography", "data mining", "database", "data structures",  "eBusiness/eCommerce", "ethics/professionalism", "formal methods", "graphics", "groupwork", "hardware/architecture", "human-computer interaction", "information systems", "introduction to IT", "literature",  "logic", "mathematics", "multimedia", "networks", "operating systems", "postgraduate/research", "programming languages", "Programming", "robotics", "school/outreach", "security", "software engineering", "study planning", "systems analysis", "webpage development", "work experience"],
                  "Theme":
                      ["ability/aptitude/understanding", "accessibility", "assessment techniques", "assessment tools", "attitudes", "cheating & plagiarism", "competitions", "credit for prior learning", "curriculum", "educational technology", "ethics/professional issues", "gender issues", "language/culture issues", "online/distance delivery", "recruitment, progression,", "pathways", "research", "teachers", "teaching/learning techniques", "teaching/learning theories & models", "teaching/learning tools", "tutors, demonstrators, mentors"
]
}

for col, options in originalOptions.items():
    df[col+"_other"] = ~df[col].isin(options)
    

In [4]:
df[df.Context_other][["Which paper","Title","Context"]].to_excel("other_contexts.xlsx")
df[df.Context_other][["Which paper","Title","Context"]]


,Which paper,Title,Context
8,2019_3294016.3294021,Data Protection and Privacy Regulations as an ...,cybersecurity
19,2019_3294016.3294026,Engaging with computer science when solving ta...,engaging non-cs pupils in cs problems
40,2020_3372356.3372362,Computer Science Degree Accreditation in the U...,professional body (BCS)
42,2020_3372356.3372363,Building an Enhanced Student Experience Reflec...,student experience & work placement
49,2020_3372356.3372366,Increasing academic diversity and inter-discip...,curriculum
51,2020_3372356.3372367,Designing a Portfolio-Oriented Curriculum usin...,curriculum
59,2020_3372356.3372371,The PASS Effect How Peer Assisted Study Sessi...,peer support
67,2021_3437914.3437971,A Repository of Resources and Exemplars for th...,cloud computing
79,2021_3437914.3437977,Supporting Early-Career Academics in the UK Co...,training for early year academics in teaching ...
84,2022_3498343.3498344,Narrowing and Stretching: Addressing the Chall...,"I dunno. It could be ""programming"", but I thin..."


In [5]:
df[df.Theme_other][["Which paper","Title","Context"]].to_excel("other_themes.xlsx")
df[df.Theme_other][["Which paper","Title","Context"]]

,Which paper,Title,Context
17,2019_3294016.3294025,A Flexible Approach to Introductory Programming,Programming
30,2019_3294016.3298736,The Institute of Coding: Addressing the UK Dig...,broad-based
40,2020_3372356.3372362,Computer Science Degree Accreditation in the U...,professional body (BCS)
44,2020_3372356.3372364,Errors and Poor Practices of Software Engineer...,software engineering
45,2020_3372356.3372364,Errors and Poor Practices of Software Engineer...,software engineering
49,2020_3372356.3372366,Increasing academic diversity and inter-discip...,curriculum
84,2022_3498343.3498344,Narrowing and Stretching: Addressing the Chall...,"I dunno. It could be ""programming"", but I thin..."
92,2022_3498343.3498348,Feedback and Engagement on an Introductory Pro...,Programming
94,2022_3498343.3498349,Co-Constructing a Community of Practice for Ea...,professional development
118,2024_3633053.3633057,Incorporating Generative AI into Software Deve...,software engineering


In [32]:
newOptions = {
              "Theme":
                [{"newoption":"generative AI",
                  "replaces": ["generative AI","generative AI? Analytic skills?"]}]
}

fixed_df = df

for col, options in newOptions.items():
    for option in options:
        fixed_df[fixed_df[col].isin(option['replaces'])] = option['newoption']


In [ ]:
# Group by year and compare raters
kappa_scores = {}

print(f"Looking at Cohen's Kappa by raters by year")
for year in df['Year'].unique():
    yearly_data = df[df['Year'] == year]

    # Pivot: one column per rater, using 'Which paper' to align their ratings
    pivot = yearly_data.pivot_table(index='Which paper', columns='Rater', values='Context', aggfunc='first')

    # Drop rows where one of the raters is missing
    pivot = pivot.dropna()

    # Ensure exactly two raters
    if pivot.shape[1] == 2:
        rater1, rater2 = pivot.columns
        ratings1 = pivot[rater1].tolist()
        ratings2 = pivot[rater2].tolist()

        kappa = cohen_kappa_score(ratings1, ratings2)
        kappa_scores[year] = kappa
        print(f"{year}: Cohen's Kappa between {rater1} and {rater2} = {kappa:.3f}")
    else:
        print(f"Year {year} skipped — expected 2 raters, found {pivot.shape[1]}")


print(f"Looking at overall Cohen's Kappa")
#df = df[df['Year'] != 2023] #excluding 2023 until duncan gives us the thing
df['Which paper'] = df['Which paper'].astype(str).str.strip().str.lower()

# Get unique raters
raters = df['Rater'].unique()

all_ratings_1 = []
all_ratings_2 = []

# For each pair of raters
for rater1, rater2 in combinations(raters, 2):
    pair_df = df[df['Rater'].isin([rater1, rater2])]
    pivot = pair_df.pivot_table(index='Which paper', columns='Rater', values='Context', aggfunc='first')
    pivot_clean = pivot.dropna()

    if pivot_clean.shape[0] == 0:
        continue  # skip pairs with no shared papers

    ratings1 = pivot_clean[rater1].tolist()
    ratings2 = pivot_clean[rater2].tolist()

    #print(ratings1)
    #print(ratings2)

    # Append to combined lists
    all_ratings_1.extend(ratings1)
    all_ratings_2.extend(ratings2)

# Compute overall Cohen's Kappa
if len(all_ratings_1) > 0:
    overall_kappa = cohen_kappa_score(all_ratings_1, all_ratings_2)
    print(f"Overall Cohen's Kappa (all pairs combined) = {overall_kappa:.3f}")
else:
    print("No overlapping ratings found between any pairs of raters.")


#Excluding OTHER

# Filter rows where Context is NOT in originalContext
excluded = df[~df['Context'].isin(originalContext)]

print(f"Number of excluded records: {len(excluded)}")
print("Excluded records with their 'Which paper':")
print(excluded[['Which paper', 'Context']])

print(f"Looking at overall Cohen's Kappa exc. Other")
df['Which paper'] = df['Which paper'].astype(str).str.strip().str.lower()

# Get unique raters
raters = df['Rater'].unique()

all_ratings_1 = []
all_ratings_2 = []

#Excluding ones that aren't in the OG themes
for rater1, rater2 in combinations(raters, 2):
    pair_df = df[df['Rater'].isin([rater1, rater2])]
    pivot = pair_df.pivot_table(index='Which paper', columns='Rater', values='Context', aggfunc='first')
    pivot_clean = pivot.dropna()

    if pivot_clean.shape[0] == 0:
        continue  # skip pairs with no shared papers

    ratings1 = pivot_clean[rater1].tolist()
    ratings2 = pivot_clean[rater2].tolist()

    # Filter pairs where both ratings are in originalContext
    filtered_pairs = [
        (r1, r2) for r1, r2 in zip(ratings1, ratings2)
        if r1 in originalContext and r2 in originalContext
    ]

    if filtered_pairs:
        filtered_ratings1, filtered_ratings2 = zip(*filtered_pairs)
        all_ratings_1.extend(filtered_ratings1)
        all_ratings_2.extend(filtered_ratings2)

# Compute overall Cohen's Kappa
if len(all_ratings_1) > 0:
    overall_kappa = cohen_kappa_score(all_ratings_1, all_ratings_2)
    print(f"Overall Cohen's Kappa (all pairs combined exc. 'other') = {overall_kappa:.3f}")
else:
    print("No overlapping ratings found between any pairs of raters.")


#NOW THEME
print("\n\n\nLOOKING AT THEME")


# Group by year and compare raters
kappa_scores = {}

print(f"Looking at Cohen's Kappa by raters by year")
for year in df['Year'].unique():
    yearly_data = df[df['Year'] == year]

    # Pivot: one column per rater, using 'Which paper' to align their ratings
    pivot = yearly_data.pivot_table(index='Which paper', columns='Rater', values='Theme', aggfunc='first')

    # Drop rows where one of the raters is missing
    pivot = pivot.dropna()

    # Ensure exactly two raters
    if pivot.shape[1] == 2:
        rater1, rater2 = pivot.columns
        ratings1 = pivot[rater1].tolist()
        ratings2 = pivot[rater2].tolist()

        kappa = cohen_kappa_score(ratings1, ratings2)
        kappa_scores[year] = kappa
        print(f"{year}: Cohen's Kappa between {rater1} and {rater2} = {kappa:.3f}")
    else:
        print(f"Year {year} skipped — expected 2 raters, found {pivot.shape[1]}")


print(f"Looking at overall Cohen's Kappa")
#df = df[df['Year'] != 2023] #excluding 2023 until duncan gives us the thing
df['Which paper'] = df['Which paper'].astype(str).str.strip().str.lower()

# Get unique raters
raters = df['Rater'].unique()

all_ratings_1 = []
all_ratings_2 = []

# For each pair of raters
for rater1, rater2 in combinations(raters, 2):
    pair_df = df[df['Rater'].isin([rater1, rater2])]
    pivot = pair_df.pivot_table(index='Which paper', columns='Rater', values='Theme', aggfunc='first')
    pivot_clean = pivot.dropna()

    if pivot_clean.shape[0] == 0:
        continue  # skip pairs with no shared papers

    ratings1 = pivot_clean[rater1].tolist()
    ratings2 = pivot_clean[rater2].tolist()

    #print(ratings1)
    #print(ratings2)

    # Append to combined lists
    all_ratings_1.extend(ratings1)
    all_ratings_2.extend(ratings2)

# Compute overall Cohen's Kappa
if len(all_ratings_1) > 0:
    overall_kappa = cohen_kappa_score(all_ratings_1, all_ratings_2)
    print(f"Overall Cohen's Kappa (all pairs combined) = {overall_kappa:.3f}")
else:
    print("No overlapping ratings found between any pairs of raters.")


#Excluding OTHER

# Filter rows where Theme is NOT in originalTheme
excluded = df[~df['Theme'].isin(originalTheme)]

print(f"Number of excluded records: {len(excluded)}")
print("Excluded records with their 'Which paper':")
print(excluded[['Which paper', 'Theme']])

print(f"Looking at overall Cohen's Kappa exc. Other")
df['Which paper'] = df['Which paper'].astype(str).str.strip().str.lower()

# Get unique raters
raters = df['Rater'].unique()

all_ratings_1 = []
all_ratings_2 = []

#Excluding ones that aren't in the OG themes
for rater1, rater2 in combinations(raters, 2):
    pair_df = df[df['Rater'].isin([rater1, rater2])]
    pivot = pair_df.pivot_table(index='Which paper', columns='Rater', values='Theme', aggfunc='first')
    pivot_clean = pivot.dropna()

    if pivot_clean.shape[0] == 0:
        continue  # skip pairs with no shared papers

    ratings1 = pivot_clean[rater1].tolist()
    ratings2 = pivot_clean[rater2].tolist()

    # Filter pairs where both ratings are in originalTheme
    filtered_pairs = [
        (r1, r2) for r1, r2 in zip(ratings1, ratings2)
        if r1 in originalTheme and r2 in originalTheme
    ]

    if filtered_pairs:
        filtered_ratings1, filtered_ratings2 = zip(*filtered_pairs)
        all_ratings_1.extend(filtered_ratings1)
        all_ratings_2.extend(filtered_ratings2)

# Compute overall Cohen's Kappa
if len(all_ratings_1) > 0:
    overall_kappa = cohen_kappa_score(all_ratings_1, all_ratings_2)
    print(f"Overall Cohen's Kappa (all pairs combined exc. 'other') = {overall_kappa:.3f}")
else:
    print("No overlapping ratings found between any pairs of raters.")